# 人工智能纳米学位
## 机器翻译项目
在此 notebook 中，标题中以**实现**开头的部分表示下面的代码块需要你提供额外的功能。请务必仔细阅读说明！

## 简介
在此 notebook 中，你将构建一个用在端到端机器翻译管道中的深度神经网络。你完成的管道将接受英文作为输入，并返回法语翻译。

- **预处理** - 将文本转换为整数序列。
- **模型** - 创建一个模型，该模型接受整数序列作为输入，并返回潜在翻译的概率分布。了解了经常用于机器翻译的基本神经网络类型后，你将自己调查并设计一个模型！
- **预测** - 在应用文本上运行该模型。

In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1',
 '/job:localhost/replica:0/task:0/device:GPU:2',
 '/job:localhost/replica:0/task:0/device:GPU:3']

In [2]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [42]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### 检查是否能访问 GPU
只有当你使用 GPU 时（例如在优达学城 Workspace 中运行或使用支持 GPU 的 AWS 实例），才需要运行以下测试。请运行下个单元格，并验证 device_type 为 GPU。
- 如果设备不是 GPU，并且你在优达学城 Workspace 中运行，则使用顶部的图标保存 Workspace，然后点击 Workspace 底部的“启用”。
- 如果设备不是 GPU，并且你使用的是 AWS 实例，则参考课堂中的云计算说明以验证你的设置步骤。

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17733913234648084983
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 31856974234
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 14550223006696765053
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:61:00.0, compute capability: 7.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 31856974234
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 2
      type: "StreamExecutor"
      strength: 1
    }
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 8660903813076889384
physical_dev

## 数据集
我们首先研究一下将用于训练和评估管道的数据集。机器翻译最常用的数据集来自 [WMT](http://www.statmt.org/)。但是，用该数据集训练神经网络需要很长的时间。我们将使用我们为此项目创建的数据集，其中包含很小的词汇表。使用该数据集，你将花费比较短的时间训练模型。
### 加载数据
数据位于 `data/small_vocab_en` 和 `data/small_vocab_fr` 中。`small_vocab_en` 文件包含英语句子，`small_vocab_fr` 文件是对应的法语翻译。请运行以下单元格，从这些文件中加载英语和法语数据。

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### 文件
`small_vocab_en` 中的每行都包含一个英语句子，`small_vocab_fr` 中的每行是相应的翻译。请查看每个文件的前两行。

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


查看这些句子后，你会发现它们已经过预处理。标点用空格代替。所有文本都转换成小写。这样你可以节省一些时间，但是文本还需要进一步预处理。
### 词汇表
该问题的复杂性由词汇表的复杂性决定。更复杂的词汇表使问题更复杂。我们来看看我们将处理的数据集的复杂性。

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])
print('{} sentences.'.format(len(english_sentences)))
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

137861 sentences.
1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


为了进行比较，_爱丽丝梦游仙境_包含 2,766 个唯一单词，一共有15,500 个单词。
## 预处理
对于此项目，你不会使用文本数据作为模型的输入，而是使用以下预处理方法将文本转换为整数序列：
1. 将单词标记化为 ID
2. 填充符号，使所有句子长度一样。

开始预处理数据...
### 标记化（实现）
要使神经网络能够对文本数据进行预测，首先需要将其转换为网络能理解的数据。“dog”等文本数据是一系列 ASCII 字符编码。因为神经网络是一系列乘法和加法运算，因此输入数据必须是数字。

我们可以将每个字符变成数字，或将每个单词变成数字，它们分别称为字符 ID 和单词 ID。字符 ID 用于为每个字符生成预测的字符级模型。单词级模型使用单词 ID 为每个单词生成文本预测。单词级模型一般学习效果更好，因为它们复杂性更低，因此我们将使用单词级模型。

使用 Keras 的 [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) 函数将每个句子转换成单词 ID 序列。在下面的单元格中使用此函数标记化 `english_sentences` 和 `french_sentences`。

运行该单元格将对样本数据运行 `tokenize`，并显示输出以进行调试。

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### 填充（实现）
批处理单词 ID 序列时，每个序列都必须长度一样。因为句子的长度不一，我们可以向序列末尾添加填充内容，使其长度相同。

确保使用 Keras 的函数 [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) 在每个序列的**末尾**添加填充内容，使所有英语序列长度相同，所有法语序列长度也相同。

In [9]:
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    print('length = ', length)
    return pad_sequences(x, maxlen=length, padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

length =  6
length =  9
length =  10
Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### 预处理管道
此项目的重点是构建神经网络架构，因此我们不需要你创建预处理管道。我们已经提供了 `preprocess` 函数的实现代码。

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

length =  15
length =  21
Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## 模型
在此部分，你将尝试各种神经网络架构。
首先，你将训练四个相对简单的架构。 
- 模型 1 是一个简单的 RNN
- 模型 2 是一个具有 Word Embedding 的 RNN
- 模型 3 是一个双向 RNN
- 模型 4 是一个可选编码器-解码器 RNN

尝试四个简单的架构后，你将构建一个效果比这四个模型更好的更深架构。
### 将 ID 变回文本
神经网络会将输入变成单词 ID，但这不是我们最终想要的格式。我们想要法语翻译。函数 `logits_to_text` 会将神经网络的 logit 与法语翻译关联起来。你将使用该函数更好地理解神经网络的输出。

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### 模型 1：RNN（实现）
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data。在此模型中，你将构建一个将英语翻译成法语的 RNN。

In [53]:
from keras.layers import GRU, Input, Dense, TimeDistributed
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    # acc: 0.6613
    model = Sequential()
    model.add(GRU(64, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.add(Activation('softmax'))
    
    # acc: 0.7566
#     model = Sequential()
#     model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
#     model.add(TimeDistributed(Dense(1024, activation='relu')))
#     model.add(Dropout(0.5))
#     model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 
    
    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_45 (GRU)                 (None, 21, 64)            12672     
_________________________________________________________________
time_distributed_39 (TimeDis (None, 21, 344)           22360     
_________________________________________________________________
activation_20 (Activation)   (None, 21, 344)           0         
Total params: 35,032
Trainable params: 35,032
Non-trainable params: 0
_________________________________________________________________
length =  21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_46 (GRU)                 (None, 21, 64)            12672     
_________________________________________________________________
time_distributed_40 (TimeDis (None, 21, 344)           22360     
___________________________________________________________

In [38]:
#new jersey est parfois calme en mois de mai et il est neigeux en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
# Print prediction(s)
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en mois de il et est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### 模型 2：嵌入word embedding（实现）
![RNN](images/embedding.png)
你已经将单词变成 ID，但是单词还有更好的表示法，称为嵌入。嵌入是单词的向量表示，像素单词在 n 维空间里更接近，其中 n 表示嵌入向量的大小。

在此模型中，你将使用嵌入创建一个 RNN 模型。

In [39]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

length =  21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 21, 256)           51200     
_________________________________________________________________
gru_39 (GRU)                 (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_28 (TimeDis (None, 21, 1024)          263168    
_________________________________________________________________
dropout_5 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_29 (TimeDis (None, 21, 345)           353625    
Total params: 1,061,977
Trainable params: 1,061,977
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 5s 47us/

In [40]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### 模型 3：双向 RNN（实现）
![RNN](images/bidirectional.png)
RNN 的一个局限之处是无法查看未来的输入，只能查看过去的输入。这时候双向递归神经网络就派上用场了，它们能够查看未来的数据。

In [45]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    # Hyperparameters
    learning_rate = 0.0003
    
    #TODO: Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                 optimizer=Adam(learning_rate),
                 metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

/home/brentweiliu/anaconda3/envs/my_notebook/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


length =  21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 21, 256)           51200     
_________________________________________________________________
gru_41 (GRU)                 (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_32 (TimeDis (None, 21, 1024)          263168    
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 21, 345)           353625    
Total params: 1,061,977
Trainable params: 1,061,977
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 5s 48us/

In [49]:
# Print prediction(s)
print("Prediction:")
print('["%s"]' % logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
["new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"]

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### 模型 4：编码器-解码器（可选）
我们来看看编码器-解码器模型。此模型由编码器和解码器组成。编码器会创建句子的矩阵表示，解码器将此矩阵当做输入，并预测翻译作为输出。

在以下单元格中创建编码器-解码器模型。

In [54]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    # Hyperparameters
    learning_rate = 0.001
    
    # Build the layers
    # acc: 0.6844
    model = Sequential()
    
    # Encoder
    model.add(GRU(256, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                 optimizer=Adam(learning_rate),
                 metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train and Print prediction(s)
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

length =  21
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_49 (GRU)                 (None, 256)               198144    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_50 (GRU)                 (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_43 (TimeDis (None, 21, 1024)          263168    
_________________________________________________________________
dropout_11 (Dropout)         (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_44 (TimeDis (None, 21, 345)           353625    
Total params: 1,208,921
Trainable params: 1,208,921
Non-trainable params: 0
_____________________________________________________

### 模型 5：自定义（实现）
请使用从之前的模型中学到的所有知识创建一个包含嵌入和双向 RNN 的模型。

In [55]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # Build the layers   
    # acc: 0.9726
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(GRU(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

Final Model Loaded


## 预测（实现）

In [56]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=25, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 15, 128)           25600     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 256)               197376    
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 21, 256)           295680    
_________________________________________________________________
time_distributed_47 (TimeDis (None, 21, 512)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 21, 512)           0         
_________________________________________________________________
time_distributed_48 (TimeDis (None, 21, 345)           176985    
Total para

## 提交
准备好提交后，请完成以下步骤：
1. 阅读[审阅标准](https://review.udacity.com/#!/rubrics/1004/view)，确保提交内容满足所有要求，以便通过审阅
2. 生成此 notebook 的 HTML 版本

  - 运行下个单元格，以便尝试自动生成（这是在 Workspace 中的推荐方法）
  - 依次转到**文件 -> 下载为 -> HTML (.html)**
  - 从 shell 终端中使用 `nbconvert` 手动生成副本

In [ ]:
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb

3. 提交项目

  - 如果是在 Workspace 中，直接点击“提交项目”按钮（位于右下角）
  
  - 否则，压缩以下文件并提交
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - 可以通过依次转到**文件 -> 下载为 -> HTML (.html)** 导出 notebook。

### 生成 html

**在运行下个单元格前先保存 notebook，以导出 HTML。** 然后提交项目。

In [ ]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

In [ ]:
['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']



## 可选增强方式

此项目侧重于学习各种机器翻译网络架构，但是没有根据最佳做法（将数据拆分为测试集和训练集）评估模型，因此模型的准确率夸大了。请使用 [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 函数创建单独的训练集和测试集，然后仅使用训练集重新训练每个模型，并使用预留的测试集评估预测准确率。“最佳”模型变了吗？